In [20]:
%tensorflow_version 2.x
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if device_name !='/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [21]:
!ls sample_data/
!nvidia-smi
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
Wed May  5 03:29:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+-------

In [18]:
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda

stat: cannot stat 'cuda': No such file or directory


In [27]:
%%cu
#include <iostream>
#include <cstdlib>

#define N (4*8)
#define THREADS_PER_BLOCK 8

using namespace std;

__global__ void add(int *a, int *b, int *c)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < N)
       c[tid]=a[tid]+b[tid];
}

int main()
{
  int a[N],b[N],c[N];
  int *d_a, *d_b, *d_c;
  int size = sizeof(int);
 
 cudaMalloc((void **)&d_a, N*size);
 cudaMalloc((void **)&d_b, N*size);
 cudaMalloc((void **)&d_c, N*size);

 for (int i=0; i<N; i++){
    a[i] = -i;
    b[i] = i * i;
 }

cudaMemcpy(d_a, &a, N*size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, &b, N*size, cudaMemcpyHostToDevice);

add<<<N/THREADS_PER_BLOCK,THREADS_PER_BLOCK>>>(d_a,d_b,d_c);

cudaMemcpy(&c,d_c,N*size,cudaMemcpyDeviceToHost);

for(int i =0; i<N; i++){
  printf("%d + %d = %d\n", a[i],b[i],c[i]);  
}

cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);

return 0;
}

0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 2
-3 + 9 = 6
-4 + 16 = 12
-5 + 25 = 20
-6 + 36 = 30
-7 + 49 = 42
-8 + 64 = 56
-9 + 81 = 72
-10 + 100 = 90
-11 + 121 = 110
-12 + 144 = 132
-13 + 169 = 156
-14 + 196 = 182
-15 + 225 = 210
-16 + 256 = 240
-17 + 289 = 272
-18 + 324 = 306
-19 + 361 = 342
-20 + 400 = 380
-21 + 441 = 420
-22 + 484 = 462
-23 + 529 = 506
-24 + 576 = 552
-25 + 625 = 600
-26 + 676 = 650
-27 + 729 = 702
-28 + 784 = 756
-29 + 841 = 812
-30 + 900 = 870
-31 + 961 = 930

